In [18]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout, GlobalAveragePooling1D, Bidirectional
from keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import numpy as np

vocab_size = 10000
embedding_dim = 64

data = pd.read_csv('test_data.csv', encoding='latin1') # 글자가 깨지지 않기 위한 인코딩 옵션

print(len(data))
print(data[:5])

7200
    constraint    mobility capability context  y
0   BESTEFFORT  STATIONARY        LOW    TEXT  1
1        DELAY  STATIONARY        LOW    TEXT  1
2  RELIABILITY  STATIONARY        LOW    TEXT  1
3   BESTEFFORT  STATIONARY     MEDIUM    TEXT  3
4        DELAY  STATIONARY     MEDIUM    TEXT  1


In [19]:
print('상태: ', data['y'].unique(), sep='\n')

상태: 
[1 3 2 4]


In [20]:
data_X = data[['constraint','mobility','capability','context']].values
data_Y = data['y'].values

print(data_X[:5])
print(data_Y[:5])

print(type(data_X))
print(type(data_Y))

data_X = data_X.tolist()
data_Y = data_Y.tolist()
print(type(data_X))
print(type(data_Y))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_X)

print(tokenizer.word_index)
print(tokenizer.word_counts)

[['BESTEFFORT' 'STATIONARY' 'LOW' 'TEXT']
 ['DELAY' 'STATIONARY' 'LOW' 'TEXT']
 ['RELIABILITY' 'STATIONARY' 'LOW' 'TEXT']
 ['BESTEFFORT' 'STATIONARY' 'MEDIUM' 'TEXT']
 ['DELAY' 'STATIONARY' 'MEDIUM' 'TEXT']]
[1 1 1 3 1]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'list'>
<class 'list'>
{'reliability': 6, 'besteffort': 3, 'high': 8, 'audio': 11, 'delay': 5, 'video': 12, 'mobile': 2, 'file': 10, 'stationary': 1, 'low': 4, 'medium': 7, 'text': 9}
OrderedDict([('besteffort', 2400), ('stationary', 3600), ('low', 2400), ('text', 1800), ('delay', 2400), ('reliability', 2400), ('medium', 2400), ('high', 2400), ('mobile', 3600), ('file', 1800), ('audio', 1800), ('video', 1800)])


In [21]:
tokenizer.texts_to_sequences(data_X)
data_X = tokenizer.texts_to_sequences(data_X)
print(data_X[:5])
print(type(data_X))

#new = [int(x) for x in data_X]
#print(new)
#new = list(map(int, data_X))
#print(new)

[[3, 1, 4, 9], [5, 1, 4, 9], [6, 1, 4, 9], [3, 1, 7, 9], [5, 1, 7, 9]]
<class 'list'>


In [22]:
(X_train, X_test, y_train, y_test) = train_test_split(data_X, data_Y, train_size=0.8, random_state=1)

In [23]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
# 훈련 데이터와 테스트 데이터에 대해서 원-핫 인코딩
print(y_train[:5])
print(y_test[:5])

[[0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]]
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [24]:
from tensorflow.keras.models import Sequential # 케라스의 Sequential()을 임포트
from tensorflow.keras.layers import Dense # 케라스의 Dense()를 임포트
from tensorflow.keras import optimizers # 케라스의 옵티마이저를 임포트

In [8]:
model=Sequential()
model.add(Dense(4, input_dim=12, activation='softmax'))
sgd=optimizers.SGD(lr=0.01)
# 학습률(learning rate, lr)은 0.01로 합니다.
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
# 옵티마이저는 경사하강법의 일종인 adam을 사용합니다.
# 손실 함수(Loss function)는 크로스 엔트로피 함수를 사용합니다.
history=model.fit(X_train,y_train, batch_size=1, epochs=200, validation_data=(X_test, y_test))
# 주어진 X와 y데이터에 대해서 오차를 최소화하는 작업을 200번 시도합니다.

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'int\'>"})'}), <class 'numpy.ndarray'>

In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6, activation='softmax')
    ])

model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 50

model.fit(np.array(X_train), y_train, epochs=num_epochs, validation_data=(np.array(X_test), y_test), verbose=2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 390       
Total params: 714,694
Trainable params: 714,694
Non-trainable params: 0
_________________________________________________________________
Train on 5760 samples, validate on 1440 samples
Epoch 1/50


ValueError: Shape mismatch: The shape of labels (received (160,)) should equal the shape of logits except for the last dimension (received (32, 6)).

In [ ]:
'''
model = Sequential()

model.add(Dense(7, input_dim=4, activation='tanh'))
model.add(Dense(4, activation='relu'))
model.add(Dense(7, activation='tanh'))
model.add(Dense(4, activation='relu'))
model.add(Dense(7, activation='tanh'))
model.add(Dense(4, activation='relu'))
model.add(Dense(7, activation='tanh'))
model.add(Dense(4, activation='relu'))
model.add(Dense(7, activation='tanh'))
model.add(Dense(4, activation='softmax'))

model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs = 50

model.fit(np.array(X_train), y_train, epochs=num_epochs, validation_data=(np.array(X_test), y_test), verbose=2)
'''

In [ ]:
from keras.layers import SpatialDropout1D

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(13, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
